In [2]:
import pandas as pd

df = pd.read_csv("FPA-FOD_reduced.csv", low_memory = False)

In [3]:
df.head()

,Unnamed: 0,pr,tmmn,tmmx,rmin,rmax,sph,srad,etr,vpd,...,Population,GDP,LATITUDE,LONGITUDE,STATE,COUNTY,FIRE_YEAR,DISCOVERY_DOY,DISCOVERY_TIME,NWCG_GENERAL_CAUSE
0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0000,51481.170,21.574287,-158.107040,HI,Oahu,2007,1,NaN,Missing data/not specified/undetermined
1,1,0.0,269.299988,281.799988,36.700001,92.099998,0.00282,129.100006,2.2,0.35,...,0.0165,47810.750,32.360830,-98.089850,TX,Erath,2007,1,NaN,Debris and open burning
2,2,0.0,276.500000,287.299988,41.799999,90.000000,0.00448,132.900009,3.6,0.47,...,2.6456,55688.996,36.487061,-121.934647,CA,Monterey,2007,1,1537.0,Misuse of fire by a minor
3,3,0.0,275.100006,286.700012,23.900000,75.200005,0.00323,144.300003,3.1,0.61,...,1.1297,47810.750,28.893870,-98.538850,TX,Atascosa,2007,1,NaN,Equipment and vehicle use
4,4,0.9,288.299988,300.299988,49.000000,98.200005,0.01116,109.500000,4.5,0.85,...,0.1981,50591.914,27.508610,-80.758610,FL,Okeechobee,2007,1,1022.0,Fireworks


In [13]:
print("Causes:")

for cause in df["NWCG_GENERAL_CAUSE"].unique():
    print("\t", cause)

Causes:
	 Missing data/not specified/undetermined
	 Debris and open burning
	 Misuse of fire by a minor
	 Equipment and vehicle use
	 Fireworks
	 Arson/incendiarism
	 Power generation/transmission/distribution
	 Railroad operations and maintenance
	 Recreation and ceremony
	 Natural
	 Smoking
	 Other causes
	 Firearms and explosives use
